In [ ]:
import os
from unittest.mock import patch, mock_open
from rss_collector.orchestration import process_feeds

def test_process_feeds():
    """
    Test the process_feeds function with mocked dependencies.
    """
    # Mock dependencies
    mock_retry_with_backoff = patch(
        "rss_collector.orchestration.retry_with_backoff",
        return_value={"entries": [{"title": "Test"}]},
    ).start()

    mock_convert_feed_to_stix = patch(
        "rss_collector.orchestration.convert_feed_to_stix",
        return_value="{}",
    ).start()

    mock_makedirs = patch("os.makedirs").start()

    mock_file = mock_open()
    with patch("builtins.open", mock_file):
        # Run the function
        feed_urls = ["https://example.com/rss"]
        process_feeds(feed_urls)

        # Assertions
        mock_retry_with_backoff.assert_called_once()
        mock_convert_feed_to_stix.assert_called_once()
        mock_makedirs.assert_called_once_with("stix_bundles", exist_ok=True)

        # Assert open is called correctly
        mock_file.assert_any_call("stix_bundles/stix_bundle_example.com.json", "w")
        mock_file().write.assert_called_once_with("{}")

    # Stop mocks
    patch.stopall()
